# Attention
Get ready for the Attention.HalfRoll experiment.
This notebook was previously called Attention.11

This notebook has code but not results used later.

In [4]:
import datetime
import numpy as np
import pandas as pd
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
BASEPATH='/home/jrm/Adjeroh/Naved/CP_80K/'
CLASS_DIR=['Output0/','Output1/','Output2/','Output3/','Output4/','Output5/',]
CLASSES=range(0,6)  # use all 6 classes
CLASSES=range(1,6)  # use smaller 5 classes
NUCFILENAME='Process100_Nucleus.csv'
ATTFILENAME='class_avg_temp.csv'

## Load data
### Attention  
A CNN was trained to assign tissue class to patches.  
The attention was scored as a probability per pixel.  
We call the score a temperature because it is used for heatmaps.  
We used bounding boxes around nuclei as generated by CellProfiler.  
We saved the average temperature per bounding box.  
Thus, we know which nuclei were attention getters.  
In the file, ObjectNumber identifies the nucleus.  
### Nuclei
Our CellProfiler pipeline called Process100 produced nucleus segmentation.  
For each detected nucleus, it generated about 600 features.  
Here, use nuclei from 80K patches from train-set WSI only.   

In [ ]:
cls=4  # testing

In [24]:
attention = pd.read_csv(BASEPATH+CLASS_DIR[cls]+ATTFILENAME)
cols={'ImageNumber':'PatchNumber',' ObjectNumber':'ObjectNumber',' AvgTemp':'AvgTemp'}
attention.rename(columns=cols,inplace=True)
attention.set_index(['PatchNumber','ObjectNumber'],inplace=True)
attention

AvgTemp
PatchNumber ObjectNumber          
631         1             0.107877
            2             0.149895
            3             0.823887
            4             0.516712
            5             0.827118
...                            ...
1337        27            0.575722
            28            0.669484
            29            0.820120
            30            0.613241
            31            0.920797

[46525 rows x 1 columns]

In [31]:
num_nuc  = len(attention)
high_temp_list = attention[attention['AvgTemp']>=.5]
low_temp_list  = attention[attention['AvgTemp']<.5]
num_hot  = len(high_temp_list)
num_cold = len(low_temp_list)
print('From the attention file, Hot: %d Cold: %d Total: %d'%(num_hot,num_cold,num_nuc))
print('The hot nuclei represent %.2f percent'%(100*num_hot/num_nuc))
print('This includes nuclei from the train and test sets.')

Hot: 23666 Cold: 22859 Total: 46525
The hot nuclei represent 50.87 percent
This includes nuclei from the train and test sets.


In [32]:
cp = CP_Util(BASEPATH+CLASS_DIR[cls])
cp.train_test_split()
nuc_df = cp.get_nuclei()
nuc_df.reset_index(inplace=True)
nuc_df.set_index(['PatchNumber','ObjectNumber'],inplace=True)
print('Features per nucleus (train set only):')
nuc_df

Num WSI in test/train sets: 2 6
Num patches in test/train sets: 813 2384
Features per nucleus (train set only):


AreaShape_Area  AreaShape_BoundingBoxArea  \
PatchNumber ObjectNumber                                              
404         1                        381                        432   
            2                        333                        420   
            3                        626                       1014   
            4                        365                        572   
            5                        360                        621   
...                                  ...                        ...   
3196        6                        491                        684   
            7                        298                        364   
            8                        235                        345   
            9                        269                        414   
3197        1                        452                        570   

                          AreaShape_BoundingBoxMaximum_X  \
PatchNumber ObjectNumber                                   
404         1                                        300   
            2                                         15   
            3                                         77   
            4                                         88   
            5                                        173   
...                                                  ...   
3196        6                                        234   
            7                                        149   
            8                                        210   
            9                                        214   
3197        1                                        263   

                          AreaShape_BoundingBoxMaximum_Y  \
PatchNumber ObjectNumber                                   
404         1                                         18   
            2                                         82   
            3                                        115   
            4                                        116   
            5                                        120   
...                                                  ...   
3196        6                                        216   
            7                                        208   
            8                                        219   
            9                                        233   
3197        1                                        103   

                          AreaShape_BoundingBoxMinimum_X  \
PatchNumber ObjectNumber                                   
404         1                                        276   
            2                                          0   
            3                                         51   
            4                                         66   
            5                                        150   
...                                                  ...   
3196        6                                        216   
            7                                        136   
            8                                        195   
            9                                        196   
3197        1                                        244   

                          AreaShape_BoundingBoxMinimum_Y  AreaShape_Center_X  \
PatchNumber ObjectNumber                                                       
404         1                                          0          287.944882   
            2                                         54            5.987988   
            3                                         76           61.797125   
            4                                         90           77.556164   
            5                                         93          159.716667   
...                                                  ...                 ...   
3196        6                                        178          224.256619   
            7                          

In [39]:
hot_nuc  = nuc_df[nuc_df.index.isin(high_temp_list.index)]
hot_patches = len(hot_nuc.index.unique(level='PatchNumber'))
hot_nuclei  = len(hot_nuc)
print('The training set includes %d hot nuclei from %d distinct patches.'%
      (hot_nuclei,hot_patches))
hot_nuc

The training set includes 18493 hot nuclei from 2332 distinct patches.


AreaShape_Area  AreaShape_BoundingBoxArea  \
PatchNumber ObjectNumber                                              
404         2                        333                        420   
            7                        381                        550   
            9                        468                        713   
            14                       282                        384   
            15                       181                        224   
...                                  ...                        ...   
3196        3                        184                        230   
            6                        491                        684   
            8                        235                        345   
            9                        269                        414   
3197        1                        452                        570   

                          AreaShape_BoundingBoxMaximum_X  \
PatchNumber ObjectNumber                                   
404         2                                         15   
            7                                         22   
            9                                        291   
            14                                       244   
            15                                        94   
...                                                  ...   
3196        3                                        229   
            6                                        234   
            8                                        210   
            9                                        214   
3197        1                                        263   

                          AreaShape_BoundingBoxMaximum_Y  \
PatchNumber ObjectNumber                                   
404         2                                         82   
            7                                        121   
            9                                        148   
            14                                       256   
            15                                       269   
...                                                  ...   
3196        3                                        115   
            6                                        216   
            8                                        219   
            9                                        233   
3197        1                                        103   

                          AreaShape_BoundingBoxMinimum_X  \
PatchNumber ObjectNumber                                   
404         2                                          0   
            7                                          0   
            9                                        260   
            14                                       220   
            15                                        80   
...                                                  ...   
3196        3                                        206   
            6                                        216   
            8                                        195   
            9                                        196   
3197        1                                        244   

                          AreaShape_BoundingBoxMinimum_Y  AreaShape_Center_X  \
PatchNumber ObjectNumber                                                       
404         2                                         54            5.987988   
            7                                         96           10.052493   
            9                                        125          274.074786   
            14                                       240          230.843972   
            15                                       253           86.309392   
...                                                  ...                 ...   
3196        3                                        105          216.934783   
            6                          

In [40]:
cold_nuc  = nuc_df[nuc_df.index.isin(low_temp_list.index)]
cold_patches = len(cold_nuc.index.unique(level='PatchNumber'))
cold_nuclei  = len(cold_nuc)
print('The training set includes %d cold nuclei from %d distinct patches.'%
      (cold_nuclei,cold_patches))
cold_nuc

The training set includes 17653 cold nuclei from 2293 distinct patches.


AreaShape_Area  AreaShape_BoundingBoxArea  \
PatchNumber ObjectNumber                                              
404         1                        381                        432   
            3                        626                       1014   
            4                        365                        572   
            5                        360                        621   
            6                        441                        575   
...                                  ...                        ...   
3196        1                        313                        364   
            2                        394                        525   
            4                        211                        260   
            5                        368                        493   
            7                        298                        364   

                          AreaShape_BoundingBoxMaximum_X  \
PatchNumber ObjectNumber                                   
404         1                                        300   
            3                                         77   
            4                                         88   
            5                                        173   
            6                                        187   
...                                                  ...   
3196        1                                        232   
            2                                        117   
            4                                        191   
            5                                         86   
            7                                        149   

                          AreaShape_BoundingBoxMaximum_Y  \
PatchNumber ObjectNumber                                   
404         1                                         18   
            3                                        115   
            4                                        116   
            5                                        120   
            6                                        122   
...                                                  ...   
3196        1                                         13   
            2                                         99   
            4                                        171   
            5                                        202   
            7                                        208   

                          AreaShape_BoundingBoxMinimum_X  \
PatchNumber ObjectNumber                                   
404         1                                        276   
            3                                         51   
            4                                         66   
            5                                        150   
            6                                        164   
...                                                  ...   
3196        1                                        204   
            2                                         96   
            4                                        178   
            5                                         69   
            7                                        136   

                          AreaShape_BoundingBoxMinimum_Y  AreaShape_Center_X  \
PatchNumber ObjectNumber                                                       
404         1                                          0          287.944882   
            3                                         76           61.797125   
            4                                         90           77.556164   
            5                                         93          159.716667   
            6                                         97          175.455782   
...                                                  ...                 ...   
3196        1                                          0          217.258786   
            2                          

In [ ]:
attention=None
nuc_df=None
high_temp_list=None
low_temp_list=None

In [41]:
print(datetime.datetime.now())
rollup = hot_nuc.groupby(['PatchNumber']).describe() ## this is slow
rollup.columns=rollup.columns.map('_'.join)  ## helps random forest code
rollup.to_csv('Hot_nucleus_rollup_class'+str(cls)+'.csv')
print(datetime.datetime.now())